In [1]:
from ecgtools import Builder
from ecgtools.parsers.cesm import parse_cesm2_le

import glob
import pandas as pd

In [8]:
parse_cesm2_le

<function ecgtools.parsers.cesm.parse_cesm2_le(file, frequency_dict=None)>

In [2]:
b = Builder(
    "/glade/campaign/cgd/cesm/CESM2-LE/timeseries/",
    depth=3,
    parsing_func=parse_cesm2_le,
    exclude_patterns=["*/hist/*", "*/rest/*", "*/restarts/*"],
    njobs=36,
)

In [4]:
b.build()

[Parallel(n_jobs=36)]: Using backend LokyBackend with 36 concurrent workers.
[Parallel(n_jobs=36)]: Done   1 tasks      | elapsed:    3.2s
[Parallel(n_jobs=36)]: Done   2 out of  17 | elapsed:    3.3s remaining:   24.9s
[Parallel(n_jobs=36)]: Done   6 out of  17 | elapsed:    3.9s remaining:    7.2s
[Parallel(n_jobs=36)]: Done  10 out of  17 | elapsed:    7.9s remaining:    5.5s
[Parallel(n_jobs=36)]: Done  14 out of  17 | elapsed:   27.3s remaining:    5.9s
[Parallel(n_jobs=36)]: Done  17 out of  17 | elapsed:  1.7min finished
[Parallel(n_jobs=36)]: Using backend LokyBackend with 36 concurrent workers.
[Parallel(n_jobs=36)]: Done  90 tasks      | elapsed:    4.7s
[Parallel(n_jobs=36)]: Done 216 tasks      | elapsed:    5.1s
[Parallel(n_jobs=36)]: Done 378 tasks      | elapsed:    5.8s
[Parallel(n_jobs=36)]: Done 720 tasks      | elapsed:    7.0s
[Parallel(n_jobs=36)]: Done 1188 tasks      | elapsed:    8.5s
[Parallel(n_jobs=36)]: Done 1728 tasks      | elapsed:   10.2s
[Parallel(n_job

Builder(root_path=PosixPath('/glade/campaign/cgd/cesm/CESM2-LE/timeseries'), extension='.nc', depth=3, exclude_patterns=['*/hist/*', '*/rest/*', '*/restarts/*'], parsing_func=<function parse_cesm2_le at 0x2ac15088fb90>, njobs=36)

In [71]:
test = None

if test:
    print('yay')
    
else:
    

In [19]:
list(b.invalid_assets.INVALID_ASSET.astype(str).values)

['/glade/campaign/cgd/cesm/CESM2-LE/timeseries/ocn/proc/tseries/month_1/sp_Qp/b.e21.BHISTcmip6.f09_g17.LE2-1161.009.pop.h.sp_Qp.199001-199912.nc',
 '/glade/campaign/cgd/cesm/CESM2-LE/timeseries/ocn/proc/tseries/month_1/spCaCO3/b.e21.BSSP370smbb.f09_g17.LE2-1301.018.pop.h.spCaCO3.209501-210012.nc',
 '/glade/campaign/cgd/cesm/CESM2-LE/timeseries/ocn/proc/tseries/month_1/PO4_diaz_uptake/b.e21.BSSP370smbb.f09_g17.LE2-1301.018.pop.h.PO4_diaz_uptake.204501-205412.nc',
 '/glade/campaign/cgd/cesm/CESM2-LE/timeseries/ocn/proc/tseries/month_1/PO4_diaz_uptake/b.e21.BSSP370smbb.f09_g17.LE2-1301.018.pop.h.PO4_diaz_uptake.206501-207412.nc',
 '/glade/campaign/cgd/cesm/CESM2-LE/timeseries/ocn/proc/tseries/month_1/PO4_diaz_uptake/b.e21.BSSP370smbb.f09_g17.LE2-1301.018.pop.h.PO4_diaz_uptake.208501-209412.nc',
 '/glade/campaign/cgd/cesm/CESM2-LE/timeseries/ocn/proc/tseries/month_1/diatFe/b.e21.BSSP370smbb.f09_g17.LE2-1301.018.pop.h.diatFe.209501-210012.nc',
 '/glade/campaign/cgd/cesm/CESM2-LE/timeseries/

In [ ]:
b.save(
    '/glade/work/mgrover/intake-esm-catalogs/cesm2-le.csv.gz',
    
    # Column name including filepath
    path_column_name='path',
    # Column name including variables
    variable_column_name='variable',
    # Data file format - could be netcdf or zarr (in this case, netcdf)
    data_format="netcdf",
    # Which attributes to groupby when reading in variables using intake-esm
    groupby_attrs=["component", "experiment", "stream"],
    # Aggregations which are fed into xarray when reading in data using intake
    aggregations=[
        {
        "type": "union",
        "attribute_name": "variable",
      },
      {
        "type": "join_existing",
        "attribute_name": "time_range",
        "options": {
          "dim": "time",
          "coords": "minimal",
          "compat": "override"
        }
      },
      {
        "type": "join_new",
        "attribute_name": "member_id",
        "options": {
          "coords": "minimal",
          "compat": "override"
        }
      }
    ]
)

In [24]:
import intake
from graphviz import Digraph

In [6]:
col = intake.open_esm_datastore('/glade/work/mgrover/intake-esm-catalogs/cesm2-le.json')

/glade/u/home/mgrover/miniconda3/envs/hires-marbl/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [1]:
# Use dask jobqueue
from ncar_jobqueue import NCARCluster
import intake

# Import a client
from dask.distributed import Client
cluster = NCARCluster(memory='10GB')
cluster.scale(20)
client = Client(cluster)

In [4]:
client

Client Scheduler: tcp://10.12.206.13:45191 Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mgrover/proxy/8787/status,Cluster Workers: 20 Cores: 40 Memory: 186.20 GiB


In [36]:
cat = col.search(component='lnd', variable='TREFMXAV', frequency='day_1', experiment_number=1251, member_id=4)

In [38]:
dsets = cat.to_dataset_dict()


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream'


In [42]:
dsets['lnd.ssp370.clm2.h6'].landunit

<xarray.DataArray 'landunit' (landunit: 62125)>
array([    0,     1,     2, ..., 62122, 62123, 62124])
Dimensions without coordinates: landunit

In [22]:
ds = xr.open_dataset('/glade/campaign/cgd/cesm/CESM2-LE/lnd/proc/tseries/day_1/TREFMXAV/b.e21.BHISTcmip6.f09_g17.LE2-1231.009.clm2.h6.TREFMXAV.19200101-19291231.nc')

In [44]:
ds.gridcell

<xarray.DataArray 'gridcell' (gridcell: 21013)>
array([    0,     1,     2, ..., 21010, 21011, 21012])
Dimensions without coordinates: gridcell

In [19]:
for var in sorted(cat.df.variable.unique()):
    print(var.lower())

In [36]:
cat = cat.search(experiment='ssp370', variable='TEMP')

In [45]:
dsets = cat.search(experiment_number=1231).to_dataset_dict()


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream'


/glade/u/home/mgrover/miniconda3/envs/hires-marbl/lib/python3.7/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/glade/u/home/mgrover/miniconda3/envs/hires-marbl/lib/python3.7/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[inde

In [25]:
# Create Digraph object - use the left to right orientation instead of vertical
dot = Digraph(format='pdf')

# Start counting at one for node numbers
num_node = 1

# Loop through the different experiments
for experiment in df.experiment.unique():
    exp_i = num_node
    dot.node(str(exp_i), label=experiment)
    num_node += 1

    # Loop through the different components in each experiment
    for forcing in df.loc[df.experiment == experiment].forcing.unique():
        comp_i = num_node
        dot.node(str(comp_i), label=forcing)
        dot.edge(str(exp_i), str(comp_i))
        num_node += 1

        # Loop through the frequency in each component within each experiment
        for experiment_id in df.loc[
            (df.experiment == experiment) & (df.forcing == forcing)
        ].experiment_number.unique():
            freq_i = num_node
            num_mems = len(df.loc[(df.experiment == experiment) & (df.forcing == forcing) &
                              (df.experiment_number == experiment_id)].member_id.unique())
            dot.node(str(freq_i), label=f'{experiment_id} \n # members: {num_mems}')
            dot.edge(str(comp_i), str(freq_i))
            num_node += 1
            ## Loop through the frequency in each component within each experiment
            #for member_id in sorted(df.loc[
            #    (df.experiment == experiment) & (df.forcing == forcing) & (df.experiment_number == experiment_id)
            #].member_id.unique()):
            #    mem_i = num_node
            #    dot.node(str(mem_i), label=member_id)
            #    dot.edge(str(freq_i), str(mem_i))
            #    num_node += 1
            #freq_i += 1
        comp_i += 1
    exp_i += 1

KeyboardInterrupt: 

In [ ]:
dot

In [ ]:
dot.save('cesm2-le')

In [ ]:
dot.ren

In [ ]:
parse_cesm2_le(test)

In [ ]:
b = Builder(
    "/glade/campaign/cgd/cesm/CESM2-LE/timeseries/atm/proc/tseries/day_1/AODVIS/",
    depth=1,
    parsing_func=parse_cesm2_le,
    exclude_patterns=["*/hist/*", "*/rest/*", "*/restarts/*", "*/DPLE_FOSI/*"],
    njobs=1,
)

In [ ]:
b.build()